<a href="https://colab.research.google.com/github/fico-aditama/fad_001_data_mining_fake_news/blob/main/detect_fake_news_with_snorkel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deteksi Berita Palsu Bidang Kesehatan dengan Snorkel dan Perceptron

Proyek ini menggunakan pendekatan **Rule-Based Labeling** dengan **Snorkel** untuk memberi label data berita kesehatan secara otomatis, kemudian melatih model **Perceptron** untuk mendeteksi berita palsu.

## 1. Instalasi Library yang Diperlukan
Jalankan perintah berikut di terminal atau cell ini untuk menginstal library yang dibutuhkan:

In [67]:
!pip install snorkel pandas scikit-learn numpy nltk --upgrade

## 2. Import Library

In [68]:
import pandas as pd
import numpy as np
from snorkel.labeling import labeling_function, PandasLFApplier
from snorkel.labeling.model import LabelModel
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import Perceptron
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import re
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## 3. Memuat dan Preprocessing Data
Data yang diberikan adalah daftar judul berita dari Kompas.com. Kita akan memuatnya ke dalam DataFrame dan melakukan preprocessing sederhana.

In [69]:
# Konstanta Label
ABSTAIN = -1
FAKE = 0
TRUE = 1

# Buat DataFrame
df = pd.DataFrame(pd.read_csv('https://docs.google.com/spreadsheets/d/11TJNGsOvSxns-Vu2dLtylK_7D6e_5zrl/export?format=csv', on_bad_lines='skip'))

# Preprocessing teks
stop_words = set(stopwords.words('indonesian'))
def preprocess_text(text):
    text = text.lower()  # Huruf kecil
    text = re.sub(r'[^a-z\s]', '', text)  # Hapus tanda baca dan angka
    text = ' '.join([word for word in text.split() if word not in stop_words])  # Hapus stop words
    return text

df['Judul Berita'] = df['Judul Berita'].apply(preprocess_text)
print("Data setelah preprocessing:")
print(df[['No', 'Judul Berita']].head())

Data setelah preprocessing:
   No                                       Judul Berita
0   1  tragedi penghapusan mandatory spending uu kese...
1   2                pindah faskes bpjs kesehatan online
2   3  uu kesehatan terbaru str dokter perawat berlak...
3   4   uu kesehatan hapus anggaran wajib minimal bidang
4   5  uu kesehatan bolehkan dokter asing berpraktik ...


## 4. Definisi Labeling Functions dengan Snorkel
Kita akan membuat aturan sederhana untuk mendeteksi potensi berita palsu berdasarkan kata kunci sensasional atau kontradiksi.

In [70]:
@labeling_function()
def sensational_keywords(x):
    keywords = ['tragedi', 'ramai', 'kontroversi', 'demo', 'tolak', 'krisis', 'polemik']
    return FAKE if any(keyword in x['Judul Berita'] for keyword in keywords) else ABSTAIN

@labeling_function()
def practical_guide(x):
    keywords = ['cara', 'langkah', 'daftar', 'tips', 'panduan']
    return TRUE if any(keyword in x['Judul Berita'] for keyword in keywords) else ABSTAIN

@labeling_function()
def official_source(x):
    keywords = ['kemenkes', 'jokowi', 'dpr', 'kpu', 'tni', 'menkes']
    return TRUE if any(keyword in x['Judul Berita'] for keyword in keywords) else ABSTAIN

@labeling_function()
def vague_or_exaggerated(x):
    keywords = ['segudang', 'ampuh', 'cepat', 'gratis', 'tanpa']
    return FAKE if any(keyword in x['Judul Berita'] for keyword in keywords) else ABSTAIN

@labeling_function()
def specific_policy(x):
    keywords = ['uu', 'ruu', 'perpres', 'aturan', 'pasal']
    return TRUE if any(keyword in x['Judul Berita'] for keyword in keywords) else ABSTAIN

lfs = [sensational_keywords, practical_guide, official_source, vague_or_exaggerated, specific_policy]

## 5. Menerapkan Labeling Functions dengan Snorkel

In [71]:
applier = PandasLFApplier(lfs=lfs)
L_train = applier.apply(df)

label_model = LabelModel(cardinality=2, verbose=True)
label_model.fit(L_train, n_epochs=1000, log_freq=100, seed=123)

df['label'] = label_model.predict(L_train)
df['Prediksi'] = df['label'].map({FAKE: 'Fake', TRUE: 'True', ABSTAIN: 'Abstain'})
print("\nHasil Labeling Awal:")
print(df[['No', 'Judul Berita', 'Prediksi']].head(10))

100%|██████████| 1000/1000 [00:05<00:00, 176.54epoch/s]


Hasil Labeling Awal:
   No                                       Judul Berita Prediksi
0   1  tragedi penghapusan mandatory spending uu kese...     True
1   2                pindah faskes bpjs kesehatan online  Abstain
2   3  uu kesehatan terbaru str dokter perawat berlak...     True
3   4   uu kesehatan hapus anggaran wajib minimal bidang     True
4   5  uu kesehatan bolehkan dokter asing berpraktik ...     True
5   6  jokowi uu kesehatan direvisi permudah dokter m...     True
6   7         ditolak idi ruu kesehatan segudang manfaat     True
7   8  uu kesehatan terbaru atur produk tembakau zat ...     True
8   9           manfaat ruu kesehatan perlindungan hukum     True
9  10  uu kesehatan terbaru abaikan pasien situasi da...     True


In [72]:
df[df['Prediksi'] != 'True'][['Judul Berita','Prediksi']].head(50)

,Judul Berita,Prediksi
1,pindah faskes bpjs kesehatan online,Abstain
10,mengaktifkan bpjs kesehatan nonaktif,Abstain
11,kemenko pmk alasan kepesertaan bpjs kesehatan,Abstain
15,komitmen aniescak imin siapkan layanan kesehatan,Abstain
16,anies misi utamanya nyapres kesetaraan bidang,Abstain
17,kongres hukum kesehatan dunia digelar batam juli,Abstain
19,adakah tes kesehatan gigi daftar sekolah intel...,Fake
21,dkpp korban alami gangguan kesehatan akibat ti...,Abstain
22,pindah faskes bpjs kesehatan langsung,Abstain
23,notifikasi denda rawat inap bpjs kesehatan,Abstain


In [76]:
df.to_csv('data_prediksi.csv', index=False)

## 6. Ekstraksi Fitur dan Pelatihan Model Perceptron
Kita akan mengubah teks menjadi fitur numerik menggunakan TF-IDF, lalu melatih model Perceptron.

In [73]:
vectorizer = TfidfVectorizer(max_features=1000, ngram_range=(1, 2))
X = vectorizer.fit_transform(df['Judul Berita']).toarray()
y = df['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
mask_train = y_train != ABSTAIN
mask_test = y_test != ABSTAIN
X_train, y_train = X_train[mask_train], y_train[mask_train]
X_test, y_test = X_test[mask_test], y_test[mask_test]

perceptron = Perceptron(max_iter=1000, tol=1e-3, random_state=42, eta0=0.1)
perceptron.fit(X_train, y_train)

y_pred = perceptron.predict(X_test)
y_test_str = ['Fake' if label == FAKE else 'True' for label in y_test]
y_pred_str = ['Fake' if label == FAKE else 'True' for label in y_pred]

print('\nAccuracy:', accuracy_score(y_test, y_pred))
print('Classification Report:\n', classification_report(y_test_str, y_pred_str, target_names=['Fake', 'True']))

# Prediksi pada seluruh data
df['Prediksi_Final'] = ['Fake' if pred == FAKE else 'True' if pred == TRUE else 'Abstain'
                       for pred in perceptron.predict(X)]
print("\nHasil Prediksi Final pada Seluruh Data:")
print(df[['No', 'Judul Berita', 'Prediksi_Final']])


Accuracy: 0.8823529411764706
Classification Report:
               precision    recall  f1-score   support

        Fake       0.67      0.67      0.67         3
        True       0.93      0.93      0.93        14

    accuracy                           0.88        17
   macro avg       0.80      0.80      0.80        17
weighted avg       0.88      0.88      0.88        17


Hasil Prediksi Final pada Seluruh Data:
      No                                       Judul Berita Prediksi_Final
0      1  tragedi penghapusan mandatory spending uu kese...           True
1      2                pindah faskes bpjs kesehatan online           Fake
2      3  uu kesehatan terbaru str dokter perawat berlak...           True
3      4   uu kesehatan hapus anggaran wajib minimal bidang           True
4      5  uu kesehatan bolehkan dokter asing berpraktik ...           True
..   ...                                                ...            ...
122  124   biaya mengurus surat keterangan sehat dita

## 7. Contoh Prediksi pada Data Baru

In [74]:
new_titles = [
    'cara cepat sembuh dari flu dengan bpjs kesehatan',
    'tragedi besar dokter asing bunuh pasien di ri',
    'jokowi umumkan uu kesehatan baru untuk rakyat',
    'ramai dokter palsu banjiri rumah sakit'
]
new_titles_cleaned = [preprocess_text(title) for title in new_titles]
X_new = vectorizer.transform(new_titles_cleaned).toarray()
predictions = perceptron.predict(X_new)

print("\nPrediksi pada Data Baru:")
for title, pred in zip(new_titles, predictions):
    label = 'Fake' if pred == FAKE else 'True'
    print(f'Judul: {title} -> Prediksi: {label}')


Prediksi pada Data Baru:
Judul: cara cepat sembuh dari flu dengan bpjs kesehatan -> Prediksi: Fake
Judul: tragedi besar dokter asing bunuh pasien di ri -> Prediksi: Fake
Judul: jokowi umumkan uu kesehatan baru untuk rakyat -> Prediksi: True
Judul: ramai dokter palsu banjiri rumah sakit -> Prediksi: True


In [77]:
new_data = {
    'Judul Berita': new_titles,
    'Prediksi': ['Fake' if pred == FAKE else 'True' for pred in predictions]
}
new_df = pd.DataFrame(new_data)
new_df.to_csv('prediksi_data_baru.csv', index=False)
print("\nHasil prediksi telah diekspor ke 'prediksi_data_baru.csv'")


Hasil prediksi telah diekspor ke 'prediksi_data_baru.csv'
